In [1]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar variables generales
estado = False
modo = 1

# Inicializar MediaPipe para reconocimiento de manos
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Variables para el modo basado en gestos
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3
tiempo_reinicio = 5
mostrar_resultado = False


chuleta_imagenes = [
    cv2.imread('SUMA.jpg'),
    cv2.imread('RESTA.jpg'),
    cv2.imread('MULTIPLICACION.jpg'),
    cv2.imread('DIVISION.jpg'),
    cv2.imread('IGUAL.jpg')
]
salir_imagen = cv2.imread('SALIR.png')

chuleta_imagenes = [cv2.resize(img, (100, 100)) for img in chuleta_imagenes]
salir_imagen = cv2.resize(salir_imagen, (100, 100))



2025-01-07 17:21:44.749655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1736270513.067868 5007804 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1736270513.156884 5008411 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736270513.198480 5008409 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [16]:
# Función para detectar gestos de manos y operar
def modo_juego_gestos(frame):
    cv2.putText(frame, f"CALCULADORA MODO 1", (400, 40), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 69, 0), 3, lineType=cv2.LINE_AA)
    cv2.putText(frame, f"(Pulsa la tecla '2' para el modo 2)", (490, 80), cv2.FONT_HERSHEY_SIMPLEX , 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

    global operacion_actual, resultado, ultimo_numero, tiempo_ultima_deteccion, mostrar_resultado

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark

            # Determinar si los dedos están levantados
            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar los dedos levantados que no sean el meñique ni el pulgar
            dedos_detectados = sum([
                dedos_levantados["indice"],
                dedos_levantados["medio"],
                dedos_levantados["anular"]
            ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]

            # Tiempo actual para el temporizador
            tiempo_actual = time.time()

            # RESTAR
            if (
                dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "-"
                resultado += ultimo_numero + "-"
                ultimo_numero = ""

            #SUMAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "+"
                resultado += ultimo_numero + "+"
                ultimo_numero = ""
            
            #MULTIPLICAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "*"
                resultado += ultimo_numero + "*"
                ultimo_numero = ""
            
            #DIVIDIR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "/"
                resultado += ultimo_numero + "/"
                ultimo_numero = ""

            #IGUAL
            elif (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                resultado += ultimo_numero
                operacion_actual += "="

                try:
                    calculo = eval(resultado)
                    resultado += f"={calculo}"
                except:
                    resultado += "=Error"
                ultimo_numero = ""
                operacion_actual = ""
                mostrar_resultado = True  # Bloquear después de "="

            elif (
                dedos_detectados >= 0
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    ultimo_numero += "0"
                elif dedos_detectados == 1:
                    ultimo_numero += "1"
                elif dedos_detectados == 2:
                    ultimo_numero += "2"
                elif dedos_detectados == 3:
                    ultimo_numero += "3"
                elif dedos_detectados == 4:
                    ultimo_numero += "4"
                elif dedos_detectados == 5:
                    ultimo_numero += "5"

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear entradas después del reinicio
    
    alto_ventana, ancho_ventana, _ = frame.shape

    # Espacio entre imágenes
    espacio_imagen = 110  # 100px de imagen + 10px de espacio

    # Calcular la posición de las imágenes en la parte inferior derecha
    x_offset = ancho_ventana - (len(chuleta_imagenes) * espacio_imagen) - 5  # 10px de margen
    y_offset = alto_ventana - 110  # 100px de altura para las imágenes

    # Coloca las imágenes en la parte inferior derecha
    for i, imagen in enumerate(chuleta_imagenes):
        # Calcula el desplazamiento horizontal
        x_pos = x_offset + i * espacio_imagen
        # Coloca cada imagen en la parte inferior derecha
        frame[y_offset:y_offset + 100, x_pos:x_pos + 100] = imagen

    # Mostrar el resultado en la pantalla
    cv2.putText(frame, f"OPERACION: ", (20, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"{resultado}{ultimo_numero}", (20, 280), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 6)



In [18]:
def modo_2(frame):
    cv2.putText(frame, f"CALCULADORA MODO 2", (400, 40), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 69, 0), 3, lineType=cv2.LINE_AA)
    cv2.putText(frame, f"(Pulsa la tecla '1' para el modo 1)", (490, 80), cv2.FONT_HERSHEY_SIMPLEX , 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)



In [17]:
# Inicializar cámara
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        break

    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):  # Simula clic en SALIR con la tecla '1'
        print("Cambiando a modo 1.")
        modo = 1
    if key == ord('2'):  # Simula clic en SALIR con la tecla '2'
        print("Cambiando a modo 2")
        modo = 2

    if modo == 1:  # Activar el modo de gestos
        modo_juego_gestos(frame)
    elif modo == 2:  # Mostrar RESTAR.jpg en modo 2
        modo_2(frame)

    cv2.imshow("Modo Interactivo", frame)

    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar MediaPipe
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Captura de video
cam = cv2.VideoCapture(0)

# Variables para mantener el estado
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3 
tiempo_reinicio = 5  # Tiempo de reinicio cuando hay espera
mostrar_resultado = False  # Bandera para bloquear la entrada después del "="


chuleta_imagenes = [
    cv2.imread('SUMA.jpg'),
    cv2.imread('RESTA.jpg'),
    cv2.imread('MULTIPLICACION.jpg'),
    cv2.imread('DIVISION.jpg'),
    cv2.imread('IGUAL.jpg')
]

chuleta_imagenes = [cv2.resize(img, (100, 100)) for img in chuleta_imagenes]

while True:
    success, img = cam.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark

            # Determinar si los dedos están levantados
            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar los dedos levantados que no sean el meñique ni el pulgar
            dedos_detectados = sum([
                dedos_levantados["indice"],
                dedos_levantados["medio"],
                dedos_levantados["anular"]
            ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]

            # Tiempo actual para el temporizador
            tiempo_actual = time.time()

            # RESTAR
            if (
                dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "-"
                resultado += ultimo_numero + "-"
                ultimo_numero = ""

            #SUMAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "+"
                resultado += ultimo_numero + "+"
                ultimo_numero = ""
            
            #MULTIPLICAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "*"
                resultado += ultimo_numero + "*"
                ultimo_numero = ""
            
            #DIVIDIR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "/"
                resultado += ultimo_numero + "/"
                ultimo_numero = ""

            #IGUAL
            elif (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                resultado += ultimo_numero
                operacion_actual += "="

                try:
                    calculo = eval(resultado)
                    resultado += f"={calculo}"
                except:
                    resultado += "=Error"
                ultimo_numero = ""
                operacion_actual = ""
                mostrar_resultado = True  # Bloquear después de "="

            elif (
                dedos_detectados >= 0
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    ultimo_numero += "0"
                elif dedos_detectados == 1:
                    ultimo_numero += "1"
                elif dedos_detectados == 2:
                    ultimo_numero += "2"
                elif dedos_detectados == 3:
                    ultimo_numero += "3"
                elif dedos_detectados == 4:
                    ultimo_numero += "4"
                elif dedos_detectados == 5:
                    ultimo_numero += "5"

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear entradas después del reinicio
    
    for i, imagen in enumerate(chuleta_imagenes):
        x_offset = 10 + i * 110  # Espacio entre imágenes
        # Coloca cada imagen en la parte superior de la ventana
        img[10:110, x_offset:x_offset + 100] = imagen

    # Mostrar el resultado en la pantalla
    cv2.putText(img, f"Operacion: {resultado}{ultimo_numero}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar imagen
    cv2.imshow("WebCam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
        break

cam.release()
cv2.destroyAllWindows()
